In [2]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [11]:
result_dir = '../Models/Results/'
files = os.listdir(result_dir)

In [12]:
for i in range(len(files)):
    files[i] = [" ".join(files[i].split('_')[:-2]), files[i].split('_')[-2:]]

In [13]:
for i in files:
    print(i[1])

['seed', '3']
['seed', '42']
['seed', '7']
['seed', '3']
['seed', '42']
['seed', '7']
['seed', '3']
['seed', '42']
['seed', '7']
['seed', '3']
['seed', '42']
['seed', '7']
['seed', '3']
['seed', '42']
['seed', '7']
['seed', '3']
['seed', '42']
['seed', '7']
['seed', '3']
['seed', '42']
['seed', '7']
['seed', '3']
['seed', '42']
['seed', '7']
['seed', '3']
['seed', '42']
['seed', '7']
['seed', '3']
['seed', '42']
['seed', '7']
['seed', '3']
['seed', '42']
['seed', '7']
['seed', '3']
['seed', '42']
['seed', '7']
['seed', '3']
['seed', '42']
['seed', '7']
['seed', '3']
['seed', '42']
['seed', '7']
['seed', '3']
['seed', '42']
['seed', '7']
['interleaved', 'dynahate']
['interleaved', 'latenthatred']
['interleaved', 'olid']
['seed', '3']
['seed', '42']
['seed', '7']
['seed', '3']
['seed', '42']
['seed', '7']
['seed', '3']
['seed', '42']
['seed', '7']
['seed', '3']
['seed', '42']
['seed', '7']
['seed', '3']
['seed', '42']
['seed', '7']
['seed', '3']
['seed', '42']
['seed', '7']
['interleaved

In [6]:
files[0]

[['bertweet', 'dynahate'], ['seed', '3']]

In [ ]:
def open_json(file):
    with open(file, encoding='utf-8') as f:
        data = json.load(f)
    return data

In [ ]:
data_dynahate = {}
data_olid = {}
data_latenthatred = {}
for file in files:
    if file == 'README.md':
        continue
    if 'dynahate' in file:
        data_dynahate[file] = open_json(result_dir+file)
    elif 'olid' in file:
        data_olid[file] = open_json(result_dir+file)
    elif 'latenthatred' in file:
        data_latenthatred[file] = open_json(result_dir+file)